## 2014 play by play data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2014 nhl data set

In [2]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20142015.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20142015.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [3]:
da['Season'] = 2014
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]
da = da.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- keep only regular season games and exclude irrelevant on-ice events

In [4]:
da = da[da['GameNumber'] <= 21230]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [5]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [6]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [7]:
da['AdvantageType'].value_counts()

EV    272425
PP     33581
SH     15482
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [8]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,CAREY PRICE,MTL,EV,2014-10-08,FAC,MTL won Neu. Zone - MTL #51 DESHARNAIS vs TOR ...,1,DAVID DESHARNAIS,TYLER BOZAK,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2014,5x5,0:00,DAVID DESHARNAIS,TYLER BOZAK
1,CAREY PRICE,MTL,EV,2014-10-08,MISS,"MTL #51 DESHARNAIS, Wrist, Wide of Net, Off. Z...",2,DAVID DESHARNAIS,NaN,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2014,5x5,0:19,DAVID DESHARNAIS,TYLER BOZAK
2,CAREY PRICE,MTL,EV,2014-10-08,BLOCK,"TOR #81 KESSEL BLOCKED BY MTL #74 EMELIN, Wri...",3,ALEXEI EMELIN,PHIL KESSEL,NaN,TOR,20001,JONATHAN BERNIER,TOR,1,6,6,2014,5x5,0:36,TOMAS PLEKANEC,TYLER BOZAK
3,CAREY PRICE,MTL,EV,2014-10-08,SHOT,"TOR ONGOAL - #42 BOZAK, Backhand, Off. Zone, 4...",4,TYLER BOZAK,NaN,NaN,TOR,20001,JONATHAN BERNIER,TOR,1,6,6,2014,5x5,0:47,TOMAS PLEKANEC,TYLER BOZAK
4,CAREY PRICE,MTL,EV,2014-10-08,HIT,"TOR #19 LUPUL HIT MTL #11 GALLAGHER, Def. Zone",5,JOFFREY LUPUL,BRENDAN GALLAGHER,NaN,TOR,20001,JONATHAN BERNIER,TOR,1,6,6,2014,5x5,0:56,TOMAS PLEKANEC,NAZEM KADRI


In [9]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [10]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [11]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [12]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [13]:
dc.shape

(3699668, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [14]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [15]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
0,2014,MTL,DAVID DESHARNAIS,82
1,2014,MTL,PA PARENTEAU,56
2,2014,MTL,MAX PACIORETTY,80
3,2014,MTL,ALEXEI EMELIN,68
4,2014,MTL,PK SUBBAN,82


In [16]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [17]:
s.shape

(1064, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [18]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [19]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [20]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [21]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [22]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
18,2014,MTL,TOMAS PLEKANEC,804,817,-13,645,685,-40
19,2014,MTL,ALEX GALCHENYUK,728,601,127,570,597,-27
20,2014,MTL,BRENDAN GALLAGHER,827,607,220,649,606,43
21,2014,MTL,TOM GILBERT,600,726,-126,560,609,-49
22,2014,MTL,ANDREI MARKOV,1012,974,38,808,814,-6


In [23]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [24]:
sa.shape

(1064, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [25]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [26]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [27]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)

In [28]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [29]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
219,2014,MTL,LARS ELLER,42,53,38,44,-6
221,2014,MTL,DALE WEISE,41,26,41,23,18
223,2014,MTL,MIKE WEAVER,16,25,16,16,0
225,2014,MTL,ANDREI MARKOV,108,70,76,50,26
227,2014,MTL,CAREY PRICE,139,127,139,94,45


In [30]:
gfa.shape

(1033, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [31]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [32]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [33]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [34]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
219,2014,TOR,TYLER BOZAK,23
392,2014,MTL,MAX PACIORETTY,37
458,2014,TOR,NAZEM KADRI,18
868,2014,MTL,TOMAS PLEKANEC,26
1318,2014,TOR,MORGAN RIELLY,8


In [35]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [36]:
dg.shape

(733, 4)

#### assists dataframe (dast)

In [37]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [38]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [39]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [40]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [41]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2014,TOR,STUART PERCY,3
5,2014,MTL,PA PARENTEAU,14
11,2014,TOR,BRANDON KOZUN,2
17,2014,MTL,ALEXEI EMELIN,11
23,2014,TOR,LEO KOMAROV,18


In [42]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [43]:
dast.shape

(841, 4)

#### shot dataframe (ds)

In [44]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [45]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
18,2014,20001,4,SHOT,TOR,TYLER BOZAK
19,2014,20001,4,SHOT,TOR,TYLER BOZAK
20,2014,20001,4,SHOT,TOR,TYLER BOZAK
21,2014,20001,4,SHOT,TOR,TYLER BOZAK
22,2014,20001,4,SHOT,TOR,TYLER BOZAK


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [46]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [47]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
18,2014,TOR,TYLER BOZAK,128.0
54,2014,TOR,BRANDON KOZUN,6.0
78,2014,MTL,BRENDAN GALLAGHER,231.0
154,2014,MTL,ALEX GALCHENYUK,142.0
189,2014,TOR,STUART PERCY,13.0


In [62]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [60]:
i = ds.copy()
i = i[i['PlayerName'].isnull()]
i.head()

,Season,TeamCode,PlayerName,Shots
1289092,2014,NSH,NaN,NaN
1697867,2014,VAN,NaN,NaN


In [61]:
ds = ds[ds['PlayerName'].notnull()]

In [63]:
ds.shape

(939, 4)

#### block dataframe (db)

In [50]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [51]:
dbl.shape

(407667, 19)

In [52]:
dbl['EventPlayer2'].isnull().sum()

12

In [53]:
dbl['EventPlayer3'].isnull().sum()

407667

In [54]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [55]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [56]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [57]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
12,2014,MTL,ALEXEI EMELIN,116.0
166,2014,TOR,JAKE GARDINER,83.0
204,2014,MTL,TOMAS PLEKANEC,53.0
248,2014,MTL,BRENDAN GALLAGHER,46.0
374,2014,MTL,PK SUBBAN,140.0


In [58]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    1
Blocks        1
dtype: int64

In [65]:
bl = dbl.copy()
bl = bl[bl['PlayerName'].isnull()]
bl.head()

,Season,TeamCode,PlayerName,Blocks
896812,2014,BUF,NaN,NaN


In [66]:
dbl = dbl[dbl['PlayerName'].notnull()]

In [67]:
dbl.shape

(1132, 4)

#### hit dataframe (dh)

In [68]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [69]:
dhit.shape

(725755, 19)

In [70]:
dhit['EventPlayer2'].isnull().sum()

0

In [71]:
dhit['EventPlayer3'].isnull().sum()

725755

In [72]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [73]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [74]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
24,2014,TOR,JOFFREY LUPUL,78
30,2014,TOR,STEPHANE ROBIDAS,141
42,2014,TOR,NAZEM KADRI,119
48,2014,TOR,ROMAN POLAK,224
72,2014,MTL,BRENDAN GALLAGHER,47


In [75]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [76]:
dhit.shape

(980, 4)

#### penalty dataframe (dpen)

In [77]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [78]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [79]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [80]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [81]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())


- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [82]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [83]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
178,2014,MTL,ALEXEI EMELIN,59.0
757,2014,MTL,TOM GILBERT,30.0
982,2014,TOR,DANIEL WINNIK,19.0
2200,2014,PHI,ZAC RINALDO,102.0
2252,2014,PHI,LUKE SCHENN,18.0


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [84]:
dp['PlayerName'].isnull().sum()

30

In [85]:
dp = dp[dp['PlayerName'].notnull()]

In [86]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [87]:
dp.shape

(879, 4)

####  miss dataframe (dmiss)

In [88]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [89]:
dmiss.shape

(326408, 19)

In [90]:
dmiss['EventPlayer2'].isnull().sum()

326408

In [91]:
dmiss['EventPlayer3'].isnull().sum()

326408

In [92]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [93]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [94]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
6,2014,MTL,DAVID DESHARNAIS,31
36,2014,TOR,PHIL KESSEL,90
66,2014,TOR,ROMAN POLAK,32
109,2014,MTL,NATHAN BEAULIEU,36
260,2014,TOR,JOFFREY LUPUL,42


In [95]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [96]:
dmiss.shape

(901, 4)

####  takeaway dataframe (dtake)

In [97]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [98]:
dtake.shape

(193546, 19)

In [99]:
dtake['EventPlayer2'].isnull().sum()

193546

In [100]:
dtake['EventPlayer3'].isnull().sum()

193546

In [101]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [102]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [103]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
99,2014,MTL,BRENDAN GALLAGHER,43
104,2014,TOR,DANIEL WINNIK,38
124,2014,TOR,TYLER BOZAK,68
130,2014,TOR,MORGAN RIELLY,41
320,2014,TOR,PETER HOLLAND,67


In [104]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [105]:
dtake.shape

(865, 4)

####  giveaway dataframe (dgive)

In [106]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [107]:
dgive.shape

(235431, 19)

In [108]:
dgive['EventPlayer2'].isnull().sum()

235431

In [109]:
dgive['EventPlayer3'].isnull().sum()

235431

In [110]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [111]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [112]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
119,2014,TOR,DION PHANEUF,43.0
199,2014,TOR,JAKE GARDINER,66.0
326,2014,MTL,PK SUBBAN,112.0
386,2014,MTL,PA PARENTEAU,13.0
434,2014,MTL,CAREY PRICE,75.0


In [113]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    1
Giveaways     1
dtype: int64

In [115]:
g = dgive.copy()
g = g[g['PlayerName'].isnull()]
g.head()

,Season,TeamCode,PlayerName,Giveaways
1420240,2014,BUF,NaN,NaN


In [116]:
dgive = dgive[dgive['PlayerName'].notnull()]

In [117]:
dgive.shape

(959, 4)

### import 2014 player shifts

In [118]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20142015.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20142015.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [119]:
dshift['Season'] = 2014
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21230]
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2014,2014-10-08,20001,1,MTL,MAX PACIORETTY,8474157,0.0,28.0,28.0
1,2014,2014-10-08,20001,1,MTL,PA PARENTEAU,8469707,0.0,28.0,28.0
2,2014,2014-10-08,20001,1,TOR,DION PHANEUF,8470602,0.0,34.0,34.0
3,2014,2014-10-08,20001,1,TOR,STEPHANE ROBIDAS,8462196,0.0,34.0,34.0
4,2014,2014-10-08,20001,1,MTL,DAVID DESHARNAIS,8471976,0.0,35.0,35.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [120]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [121]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [122]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2014,2014-10-08,20001,1,MTL,MAX PACIORETTY,8474157,0.0,28.0,28.0,1097.0,18.28
1,2014,2014-10-08,20001,1,MTL,PA PARENTEAU,8469707,0.0,28.0,28.0,1010.0,16.83
2,2014,2014-10-08,20001,1,TOR,DION PHANEUF,8470602,0.0,34.0,34.0,1411.0,23.52
3,2014,2014-10-08,20001,1,TOR,STEPHANE ROBIDAS,8462196,0.0,34.0,34.0,1203.0,20.05
4,2014,2014-10-08,20001,1,MTL,DAVID DESHARNAIS,8471976,0.0,35.0,35.0,1039.0,17.32


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [123]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(0)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2014,2014-10-08,20001,1,MTL,MAX PACIORETTY,8474157,0.0,28.0,28.0,1097.0,18.28,93991.0,1567.0
1,2014,2014-10-08,20001,1,MTL,PA PARENTEAU,8469707,0.0,28.0,28.0,1010.0,16.83,50360.0,839.0
2,2014,2014-10-08,20001,1,TOR,DION PHANEUF,8470602,0.0,34.0,34.0,1411.0,23.52,99614.0,1660.0
3,2014,2014-10-08,20001,1,TOR,STEPHANE ROBIDAS,8462196,0.0,34.0,34.0,1203.0,20.05,53257.0,888.0
4,2014,2014-10-08,20001,1,MTL,DAVID DESHARNAIS,8471976,0.0,35.0,35.0,1039.0,17.32,84865.0,1414.0


- keep only observation per player for the season

In [124]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2014,MTL,MAX PACIORETTY,1567.0
1,2014,MTL,PA PARENTEAU,839.0
2,2014,TOR,DION PHANEUF,1660.0
3,2014,TOR,STEPHANE ROBIDAS,888.0
4,2014,MTL,DAVID DESHARNAIS,1414.0


In [126]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [127]:
dshift.shape

(1064, 4)

### import 2014 player positions

In [152]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20142015.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20142015.csv')

In [153]:
dpos.head()

,First Name,Last Name,End Team,Pos,FName,LName,PlayerName
0,Justin,Abdelkader,DET,LW,JUSTIN,ABDELKADER,JUSTIN ABDELKADER
1,Will,Acton,EDM,C/RW,WILL,ACTON,WILL ACTON
2,Luke,Adam,CBJ,C,LUKE,ADAM,LUKE ADAM
3,Craig,Adams,PIT,RW,CRAIG,ADAMS,CRAIG ADAMS
4,Andrew,Agozzino,COL,LW,ANDREW,AGOZZINO,ANDREW AGOZZINO


In [154]:
dpos['Season'] = 2014
dpos= dpos.rename(columns={ 'End Team': 'TeamCode', 'Pos': 'Position'})
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2014,DET,JUSTIN ABDELKADER,LW
1,2014,EDM,WILL ACTON,C/RW
2,2014,CBJ,LUKE ADAM,C
3,2014,PIT,CRAIG ADAMS,RW
4,2014,COL,ANDREW AGOZZINO,LW


In [155]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [156]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else 'D' if x['Position'] == 'D/RW' else 'C' if x['Position'] == 'C/D' else 'W' if x['Position'] == 'RW/D' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    317
D    307
C    258
G     92
Name: Position, dtype: int64

In [157]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2014,DET,JUSTIN ABDELKADER,W
1,2014,EDM,WILL ACTON,C
2,2014,CBJ,LUKE ADAM,C
3,2014,PIT,CRAIG ADAMS,W
4,2014,COL,ANDREW AGOZZINO,W


In [158]:
dpos.shape

(974, 4)

### merge data frames

- merge games played, goals and assists.

In [159]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')

- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [160]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
#dd['Position'] = dd.apply(lambda x: 'D' if x['PlayerName'] == 'MICHAEL KOSTKA' else 'C' if x['PlayerName'] == 'DANNY BRIERE' else x['Position'] , axis=1)
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2014,MTL,DAVID DESHARNAIS,82,C,14.0,34.0
1,2014,MTL,PA PARENTEAU,56,W,8.0,14.0
2,2014,MTL,MAX PACIORETTY,80,W,37.0,30.0
3,2014,MTL,ALEXEI EMELIN,68,D,3.0,11.0
4,2014,MTL,PK SUBBAN,82,D,15.0,45.0


In [161]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         331
Assists       223
dtype: int64

In [162]:
fix = dd.copy()
fix = fix[fix['Position'].isnull()]
fix.head(20)

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists


- fill in goals and assists with 0 for players who had no points.

In [163]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2014,MTL,DAVID DESHARNAIS,82,C,14.0,34.0,48.0
1,2014,MTL,PA PARENTEAU,56,W,8.0,14.0,22.0
2,2014,MTL,MAX PACIORETTY,80,W,37.0,30.0,67.0
3,2014,MTL,ALEXEI EMELIN,68,D,3.0,11.0,14.0
4,2014,MTL,PK SUBBAN,82,D,15.0,45.0,60.0


In [164]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [165]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2014,MTL,DAVID DESHARNAIS,82,C,14.0,34.0,48.0,78.0,37.0,60.0,37.0,23.0,76.0,733,631,102,587,630,-43,48.0,43.0,24.0,27.0,20.0,31.0,1414.0
1,2014,MTL,PA PARENTEAU,56,W,8.0,14.0,22.0,37.0,25.0,27.0,25.0,2.0,89.0,402,322,80,314,322,-8,19.0,32.0,30.0,18.0,13.0,35.0,839.0
2,2014,MTL,MAX PACIORETTY,80,W,37.0,30.0,67.0,90.0,41.0,72.0,29.0,43.0,265.0,816,730,86,656,633,23,38.0,106.0,32.0,43.0,43.0,108.0,1567.0
3,2014,MTL,ALEXEI EMELIN,68,D,3.0,11.0,14.0,44.0,56.0,44.0,39.0,5.0,40.0,470,723,-253,470,589,-119,116.0,204.0,59.0,16.0,56.0,27.0,1363.0
4,2014,MTL,PK SUBBAN,82,D,15.0,45.0,60.0,120.0,76.0,86.0,61.0,25.0,155.0,1105,979,126,869,845,24,140.0,97.0,74.0,29.0,112.0,86.0,2149.0


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [166]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [167]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [168]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [169]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep sesaon total variables and scaled variables

In [170]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2014,MTL,DAVID DESHARNAIS,C,82,14.0,34.0,48.0,78.0,37.0,60.0,37.0,23.0,76.0,733,631,102,587,630,-43,48.0,43.0,24.0,27.0,20.0,31.0,1414.0,0.009901,0.024045,0.033946,0.055163,0.026167,0.042433,0.026167,0.016266,0.053748,0.518388,0.446252,0.072136,0.415134,0.445545,-0.030410,0.033946,0.030410,0.016973,0.019095,0.014144,0.021924
1,2014,MTL,PA PARENTEAU,W,56,8.0,14.0,22.0,37.0,25.0,27.0,25.0,2.0,89.0,402,322,80,314,322,-8,19.0,32.0,30.0,18.0,13.0,35.0,839.0,0.009535,0.016687,0.026222,0.044100,0.029797,0.032181,0.029797,0.002384,0.106079,0.479142,0.383790,0.095352,0.374255,0.383790,-0.009535,0.022646,0.038141,0.035757,0.021454,0.015495,0.041716
2,2014,MTL,MAX PACIORETTY,W,80,37.0,30.0,67.0,90.0,41.0,72.0,29.0,43.0,265.0,816,730,86,656,633,23,38.0,106.0,32.0,43.0,43.0,108.0,1567.0,0.023612,0.019145,0.042757,0.057435,0.026165,0.045948,0.018507,0.027441,0.169113,0.520740,0.465858,0.054882,0.418634,0.403957,0.014678,0.024250,0.067645,0.020421,0.027441,0.027441,0.068922
3,2014,MTL,ALEXEI EMELIN,D,68,3.0,11.0,14.0,44.0,56.0,44.0,39.0,5.0,40.0,470,723,-253,470,589,-119,116.0,204.0,59.0,16.0,56.0,27.0,1363.0,0.002201,0.008070,0.010271,0.032282,0.041086,0.032282,0.028613,0.003668,0.029347,0.344828,0.530448,-0.185620,0.344828,0.432135,-0.087307,0.085106,0.149670,0.043287,0.011739,0.041086,0.019809
4,2014,MTL,PK SUBBAN,D,82,15.0,45.0,60.0,120.0,76.0,86.0,61.0,25.0,155.0,1105,979,126,869,845,24,140.0,97.0,74.0,29.0,112.0,86.0,2149.0,0.006980,0.020940,0.027920,0.055840,0.035365,0.040019,0.028385,0.011633,0.072127,0.514193,0.455561,0.058632,0.404374,0.393206,0.011168,0.065147,0.045137,0.034435,0.013495,0.052117,0.040019


In [171]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2014_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2014_player_stats.csv', index='False', sep=',')